In [1]:
%matplotlib notebook

import os
import glob
import math
import pandas as pd
import numpy as np
import itertools
import random

import matplotlib.pyplot as plt
import matplotlib.animation as animation
import matplotlib.patches as patches

import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from keras.utils import np_utils
from keras.utils.vis_utils import model_to_dot
from keras.callbacks import EarlyStopping
from IPython.display import SVG

import tensorflow_probability as tfp


from sklearn.metrics import classification_report, confusion_matrix
from sklearn import preprocessing
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer, normalize
from sklearn.preprocessing import MinMaxScaler
import warnings

warnings.filterwarnings('ignore')

from ipywidgets import FloatProgress
from IPython.display import display

fig_width = 12
plt.rcParams["font.size"] = 7

Using TensorFlow backend.


In [2]:
def join_res_data(mycsvdir):
    
    csvfiles = glob.glob(os.path.join(mycsvdir, '*.csv'))
    dataframes = []
    
    for csvfile in csvfiles:
        #print(csvfile)
        df = pd.read_csv(csvfile)
        print(df)
        dataframes.append(df)
    result = pd.concat(dataframes, ignore_index=True)
    
    return result

In [3]:
def get_activity_index(act_dict):
    labels = []
    indices = []
    for k,v in act_dict.items():
        indices.append(k)
        labels.append(v)
    return labels, indices

def plot_confusion_matrix(cm, fig, cm_plt, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)

    simg = cm_plt.imshow(cm, interpolation='nearest', cmap=cmap)
    cm_plt.set_title(title)
    tick_marks = np.arange(len(classes))
    cm_plt.set_xticks(tick_marks)
    cm_plt.set_yticks(tick_marks)
    cm_plt.set_xticklabels(classes)
    cm_plt.set_yticklabels(classes)
    cm_plt.tick_params(axis='both', which='major', labelsize=10)
    cm_plt.tick_params(axis='both', which='minor', labelsize=8)
    fig.colorbar(simg)
    
    
    plt.setp(cm_plt.get_xticklabels(), rotation=45, horizontalalignment='right')

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    cm_plt.set_ylabel('True label')
    cm_plt.set_xlabel('Predicted label')
    #cm_plt.tight_layout()

In [4]:
simclass_df = pd.read_csv(r'G:\JaimeMorales\Codes\MGA-net\datasets\OpenPack\labels.csv')
simclass_d = simclass_df['operation_l'].to_dict()
#del simclass_d[8]
#del simclass_d[10]
target_names,indices = get_activity_index(simclass_d)

In [5]:
simclass_d

{0: 'Picking',
 1: 'RelocateItemLabel',
 2: 'AssembleBox',
 3: 'InsertItems',
 4: 'CloseBox',
 5: 'AttachBoxLabel',
 6: 'ScanLabel',
 7: 'AttachShippingLabel',
 8: 'PutOnRack',
 9: 'Fill-outOrder'}

In [6]:
#simclass_d = {0:jumbi1,1:neji1,2:neji2,3:neji3,4:neji4,5:neji5,6:neji6,7:neji7,8:neji8,9:button,10:haraidashi}

In [7]:
indices

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [8]:
#day_res_df = join_res_data(r'C:\Users\Jaime\Documents\JaimeMorales\Codes\HAR_acc\acc2\results\LSTM\3')
#day_res_df = join_res_data(r'C:\Users\Jaime\Documents\JaimeMorales\Codes\HAR_acc\acc2\results\LSTM\4')
#day_res_df = join_res_data(r'C:\Users\Jaime\Documents\JaimeMorales\Codes\HAR_acc\acc2\results\LSTM\5')
#day_res_df = join_res_data(r'C:\Users\Jaime\Documents\JaimeMorales\Codes\HAR_acc\acc2\results\LSTM\6')

In [9]:
#day_res_df = join_res_data(r'C:/Users/Jaime/Documents/JaimeMorales/Codes/HAR_acc/acc2/results/Pure_Unet\3')
#day_res_df = join_res_data(r'C:/Users/Jaime/Documents/JaimeMorales/Codes/HAR_acc/acc2/results/Pure_Unet\4')
#day_res_df = join_res_data(r'C:/Users/Jaime/Documents/JaimeMorales/Codes/HAR_acc/acc2/results/Pure_Unet\5')
#day_res_df = join_res_data(r'C:/Users/Jaime/Documents/JaimeMorales/Codes/HAR_acc/acc2/results/Pure_Unet\6')

In [10]:
day_res_df = join_res_data(r'G:\JaimeMorales\Codes\MGA-net\datasets\OpenPack\Results\MGA-Net\U0108')

ValueError: No objects to concatenate

In [ ]:
#day_res_df = join_res_data(r'C:\Users\Jaime\Documents\JaimeMorales\Codes\HAR_acc\acc2\results\Test-17-08\Self-Att-con\3')
#day_res_df = join_res_data(r'C:\Users\Jaime\Documents\JaimeMorales\Codes\HAR_acc\acc2\results\Test-17-08\Self-Att-con\4')
#day_res_df = join_res_data(r'C:\Users\Jaime\Documents\JaimeMorales\Codes\HAR_acc\acc2\results\Test-17-08\Self-Att-con\5')
#day_res_df = join_res_data(r'C:\Users\Jaime\Documents\JaimeMorales\Codes\HAR_acc\acc2\results\Test-17-08\Self-Att-con\6')

In [ ]:
#day_res_df = join_res_data(r'C:\Users\Jaime\Documents\JaimeMorales\Codes\HAR_acc\acc2\results\extra\1_Pure-Unet\3')
#day_res_df = join_res_data(r'C:\Users\Jaime\Documents\JaimeMorales\Codes\HAR_acc\acc2\results\extra\1_Pure-Unet\4')
#day_res_df = join_res_data(r'C:\Users\Jaime\Documents\JaimeMorales\Codes\HAR_acc\acc2\results\extra\1_Pure-Unet\5')
#day_res_df = join_res_data(r'C:\Users\Jaime\Documents\JaimeMorales\Codes\HAR_acc\acc2\results\Testing-6-4\Pure_Unet\3')

In [ ]:
#day_res_df = join_res_data(r'C:\Users\Jaime\Documents\JaimeMorales\Codes\HAR_acc\acc2\results\Test-27-07\LSTM-Att\3')
#day_res_df = join_res_data(r'C:\Users\Jaime\Documents\JaimeMorales\Codes\HAR_acc\acc2\results\Test-27-07\LSTM-Att\4')
#day_res_df = join_res_data(r'C:\Users\Jaime\Documents\JaimeMorales\Codes\HAR_acc\acc2\results\Test-27-07\LSTM-Att\5')
#day_res_df = join_res_data(r'C:\Users\Jaime\Documents\JaimeMorales\Codes\HAR_acc\acc2\results\Test-27-07\LSTM-Att\6')

In [ ]:
#day_res_df = pd.read_csv(r'G:\JaimeMorales\Codes\HAR_acc\acc2\results\Round_2\CNN-LSTM\res_3_1.csv')

In [ ]:
day_res_df

In [ ]:
day_res_df = day_res_df[day_res_df['l_val']!=10]
#day_res_df = day_res_df[day_res_df['l_val']!=8]

In [ ]:
day_res_df = day_res_df[day_res_df['predicted_label']!=10]
#day_res_df = day_res_df[day_res_df['predicted_label']!=8]

In [ ]:
day_res_df[day_res_df['predicted_label']==10]

In [ ]:
day_res_df[day_res_df['l_val']==10]

In [ ]:
day_res_df

In [ ]:
day_res_df.to_csv(r'G:\JaimeMorales\Codes\MGA-net\datasets\OpenPack\Results\MGA-Net-with-Bi-GRU\U0101_res.csv',index=False)

In [ ]:
testYYY = np.array(list(day_res_df['l_val']))
predict1 = np.array(list(day_res_df['predicted_label']))

In [ ]:
testYYY.shape

In [ ]:
predict1.shape

In [ ]:
cnf_matrix = confusion_matrix(testYYY, predict1)
fig, (tr_plt,cm_plt) = plt.subplots(nrows=2, ncols=1, figsize=(6, 6))
plot_confusion_matrix(cnf_matrix, fig, cm_plt, classes=target_names)
fig.show()
result = classification_report(testYYY, predict1, labels=indices, target_names=target_names)
print(result)